In [1]:
using Flight.FlightCore.Systems
using Flight.FlightCore.Plotting

using Flight.FlightPhysics.Geodesy
using Flight.FlightPhysics.Kinematics
using Flight.FlightComponents.Aircraft
using Flight.FlightComponents.Control
using Flight.FlightAircraft.C172FBW
using Flight.FlightAircraft.C172

using UnPack
using ControlSystems
using RobustAndOptimalControl
using ComponentArrays
using LinearAlgebra

In [26]:
ac = Cessna172FBWBase(NED()) |> System #linearization requires NED kinematics

design_condition = C172.TrimParameters(
    Ob = Geographic(LatLon(), HOrth(1000)),
    EAS = 40.0,
    γ_wOb_n = 0.0,
    x_fuel = 0.5,
    flaps = 0.0,
    payload = C172.PayloadU(m_pilot = 75, m_copilot = 75, m_baggage = 50))

P_lss_full = Control.LinearStateSpace(ac, design_condition; model = :lon);
P_nss_full = named_ss(P_lss_full);

x_labels = [:q, :θ, :v_x, :v_z, :α_filt, :ω_eng, :ele_v, :ele_p, :thr_v, :thr_p]
u_labels = [:elevator_cmd, :throttle_cmd]
y_labels = [:q, :θ, :α, :EAS, :TAS, :f_x, :f_z, :γ, :c, :ω_eng, :v_D]
# x_labels_pitch = [:q, :θ, :v_x, :v_z, :α_filt, :ele_v, :ele_p]
# u_labels_pitch = [:elevator_cmd]
# y_labels_pitch = [:q, :θ, :α, :EAS]

P_lss = submodel(P_lss_full; x = x_labels, u = u_labels, y = y_labels)
P_nss = named_ss(P_lss);

## 1. Longitudinal Dynamics Regulator

In [3]:
dampreport(P_nss_full)
dampreport(P_nss)

|        Pole        |   Damping     |   Frequency   |   Frequency   | Time Constant |
|                    |    Ratio      |   (rad/sec)   |     (Hz)      |     (sec)     |
+--------------------+---------------+---------------+---------------+---------------+


| -0.000369          |  1            |  0.000369     |  5.87e-05     |  2.71e+03     |


| -0.0176 ±  0.293im |  0.0601       |  0.293        |  0.0467       |  56.8         |
| -4.42   ±   5.14im |  0.652        |  6.78         |  1.08         |  0.226        |
| -9.72              |  1            |  9.72         |  1.55         |  0.103        |
| -18.8   ±   25.1im |  0.6          |  31.4         |  5            |  0.0531       |
| -18.8   ±   25.1im |  0.6          |  31.4         |  5            |  0.0531       |
| -48.1              |  1            |  48.1         |  7.65         |  0.0208       |
|        Pole        |   Damping     |   Frequency   |   Frequency   | Time Constant |
|                    |    Ratio      |   (rad/sec)   |     (Hz)      |     (sec)     |
+--------------------+---------------+---------------+---------------+---------------+
| -0.0178 ±  0.291im |  0.061        |  0.292        |  0.0465       |  56.2         |
| -4.42   ±   5.14im |  0.652        |  6.78         |  1.08         |  0.226        |
| -9.72              |  1            |  9.7

We see that removing $h$ as a state eliminates an extremely slow pole, probably related to the effect of altitude on engine output. The remaining poles are virtually unaffected. Also, it significantly improves the condition of the dynamics matrix:

In [4]:
cond(P_nss_full.A) |> display
cond(P_nss.A) |> display

7.656238481914343e7

440485.80842732603

Thus, we will take the MIMO system without $h$ as our plant baseline. Let's start by designing a full-state feedback regulator for the MIMO system.

In [27]:
controllability(P_nss)

(iscontrollable = true, ranks = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10], sigma_min = [0.0026382090058166643, 1.2392048041221948, 1.2392048041221948, 1.2392048041221948, 1.2392048041221948, 0.4879588010303914, 0.4425611178586627, 0.4425611178586627, 0.040256617772433265, 0.040256617772433265])

The open-loop system is:
$$\Delta \dot{x} = A \Delta x + B \Delta u$$
$$\Delta y = C \Delta x$$

The regulator control law is simply $\Delta u = -K \Delta x$, which results in a homogeneous closed-loop system:
$$\Delta \dot{x} = (A -BK) \Delta x$$
$$\Delta y = C \Delta x$$

To retain control over the system, we instead use the control law $\Delta u = -K \Delta x + \Delta u_{ext}$, which gives:
$$\Delta \dot{x} = (A - BK) \Delta x + B \Delta u_{ext}$$
$$\Delta y = C \Delta x$$

This system has the same stability properties as the homogeneous one.

## FROM HERE. Scale v_x and v_z costs with v norm

In [31]:
@unpack x0, u0 = P_lss
@unpack v_x, v_z = x0
v_norm = norm([v_x, v_z])

#the states we care about are q, θ, v_x, v_z (equivalent to V and α)
diagQ = ComponentVector(q = 1, θ = 1, v_x = 1/v_norm, v_z = 1/v_norm, α_filt = 0, ω_eng = 0, ele_v = 0, ele_p = 0, thr_v = 0, thr_p = 0)
diagR = ComponentVector(elevator_cmd = 1, thr_cmd = 1)
Q = ComponentMatrix(diagm(diagQ), Axis(x_labels), Axis(x_labels))
R = ComponentMatrix(diagm(diagR), Axis(x_labels), Axis(x_labels))

#feedback gain matrix
C_fbk = lqr(P_nss, Q, R)
display(C_fbk)

C_fbk_ss = named_ss(ss(C_fbk); u = x_labels, y = [:elevator_fbk, :throttle_fbk])


elevator_sum = sumblock("elevator_cmd = elevator_fwd - elevator_fbk")
throttle_sum = sumblock("throttle_cmd = throttle_fwd- throttle_fbk")
connections = vcat(Pair.(x_labels, x_labels), Pair.(C_ss.y, C_ss.y))

P_nss_fbk = connect([elevator_sum, throttle_sum, P_nss, C_fbk_ss], connections; w1 = [:elevator_fwd, :throttle_fwd])

# lon_sum = sumblock("q_err = q_dmd - q")
# thr_q_MIMO = connect([q2e_sum, C_q2e, thr_ele_MIMO], [:q_err=>:q_err, :q=>:q, :elevator_cmd=>:elevator_cmd], w1 = [:throttle_cmd, :q_dmd], z1 = thr_ele_MIMO.y)
# #add pitch regulation to the reduced pitch dynamics
# A_pitch_sas = lss_pitch.A - lss_pitch.B * K_pitch
# nss_pitch_sas = named_ss(ss(A_pitch_sas, lss_pitch.B, lss_pitch.C, 0), u = [:sas_pitch_cmd], x = nss_pitch.x, y = nss_pitch.y);

# dampreport(nss_pitch)
# dampreport(nss_pitch_sas)
# e2q = nss_pitch[:q, :elevator_cmd]
# e2q_sas = nss_pitch_sas[:q, :sas_pitch_cmd]
# e2θ = nss_pitch[:θ, :elevator_cmd]
# e2θ_sas = nss_pitch_sas[:θ, :sas_pitch_cmd]
# step(e2q, 10) |> plot
# step(e2q_sas, 10) |> plot!

# A_pitch_sas[:θ, :]

2×10 Matrix{Float64}:
 0.644496    2.49538   -0.0981694  -0.0523089  …  -7.31752e-5   -0.00147389
 0.0320388  -0.532192   0.0771073   0.012023       0.000169545   0.00640854

┌ Warning: The following inputs were unconnected [:elevator_cmd, :throttle_cmd], ignore this warning if you rely on prefix matching
└ @ RobustAndOptimalControl C:\Users\Miguel\.julia\packages\RobustAndOptimalControl\CHKAo\src\named_systems2.jl:392
┌ Warning: The following outputs were unconnected Any[:elevator_cmd, :throttle_cmd, :α, :EAS, :TAS, :f_x, :f_z, :γ, :c, :v_D], ignore this warning if you rely on prefix matching
└ @ RobustAndOptimalControl C:\Users\Miguel\.julia\packages\RobustAndOptimalControl\CHKAo\src\named_systems2.jl:394


ErrorException: The indexed NamedSystem has no signal named Colon(), available names are [:elevator_cmd, :throttle_cmd, :q, :θ, :α, :EAS, :TAS, :f_x, :f_z, :γ, :c, :ω_eng, :v_D, :elevator_fbk, :throttle_fbk]

In [7]:
#build a gain matrix for the complete longitudinal dynamics and assign the
#components corresponding to the gain matrix computed from the reduced pitch
#dynamics
K_pitch_full = fill!(lss_lon.u0 * lss_lon.x0', 0)
K_pitch_full[:elevator_cmd, x_labels_pitch] = K_pitch
A_pitch_full_sas = lss_lon.A - lss_lon.B * K_pitch_full;

#add pitch regulation to the full longitudinal dynamics
nss_lon_sas = named_ss(ss(A_pitch_full_sas, lss_lon.B, lss_lon.C, 0), u = [:sas_pitch_cmd, :throttle_cmd], x = nss_lon.x, y = nss_lon.y);

dampreport(nss_pitch)
dampreport(nss_lon_sas)
e2q = nss_lon[:q, :elevator_cmd]
e2q_sas = nss_lon_sas[:q, :sas_pitch_cmd]
e2θ = nss_lon[:θ, :elevator_cmd]
e2θ_sas = nss_lon_sas[:θ, :sas_pitch_cmd]
step(e2q, 10) |> plot
step(e2q_sas, 10) |> plot!

UndefVarError: UndefVarError: `K_pitch` not defined

In [8]:
bodeplot(e2q)

UndefVarError: UndefVarError: `e2q` not defined

In [9]:
bodeplot(e2q_sas)

UndefVarError: UndefVarError: `e2q_sas` not defined

In [10]:

P_e2q = e2q_sas
C_q2e = named_ss(ss(20 * tf(1, [1, 0, 0])), :C_q2e; u = :q_err, y = :sas_pitch_cmd);
L_q2e = series(C_q2e, P_e2q)
T_q2e = output_comp_sensitivity(P_e2q, C_q2e)
# CS_q2e = G_CS(P_e2q, C_q2e)

marginplot(L_q2e)
# marginplot(CS_q2e)
step(T_q2e, 10) |> plot

UndefVarError: UndefVarError: `e2q_sas` not defined